# DeepLabV3: Toward Data Science Example
Source: https://towardsdatascience.com/transfer-learning-for-segmentation-using-deeplabv3-in-pytorch-f770863d6a42

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Imports
import sys
sys.path.append('/jcDataStore/Projects/NeuroTK-Dash')

from pathlib import Path
from typing import Any, Callable, Optional
import numpy as np
from PIL import Image
from glob import glob
import cv2 as cv
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import f1_score, roc_auc_score
import time
import csv
import copy

import os
from os.path import join, isfile

from neurotk import imread, imwrite
from neurotk.utils import create_dirs

import torch
from torch.utils.data import DataLoader

from torchvision.datasets.vision import VisionDataset
from torchvision.models.segmentation.deeplabv3 import (
    DeepLabHead, DeepLabV3_ResNet101_Weights
)
from torchvision import models

In [ ]:
"""
Author: Manpreet Singh Minhas
Contact: msminhas at uwaterloo ca
"""
from pathlib import Path
from typing import Any, Callable, Optional

import numpy as np
from PIL import Image
from torchvision.datasets.vision import VisionDataset


class SegmentationDataset(VisionDataset):
    """A PyTorch dataset for image segmentation task.
    The dataset is compatible with torchvision transforms.
    The transforms passed would be applied to both the Images and Masks.
    """
    def __init__(self,
                 root: str,
                 image_folder: str,
                 mask_folder: str,
                 transforms: Optional[Callable] = None,
                 seed: int = None,
                 fraction: float = None,
                 subset: str = None,
                 image_color_mode: str = "rgb",
                 mask_color_mode: str = "grayscale") -> None:
        """
        Args:
            root (str): Root directory path.
            image_folder (str): Name of the folder that contains the images in the root directory.
            mask_folder (str): Name of the folder that contains the masks in the root directory.
            transforms (Optional[Callable], optional): A function/transform that takes in
            a sample and returns a transformed version.
            E.g, ``transforms.ToTensor`` for images. Defaults to None.
            seed (int, optional): Specify a seed for the train and test split for reproducible results. Defaults to None.
            fraction (float, optional): A float value from 0 to 1 which specifies the validation split fraction. Defaults to None.
            subset (str, optional): 'Train' or 'Test' to select the appropriate set. Defaults to None.
            image_color_mode (str, optional): 'rgb' or 'grayscale'. Defaults to 'rgb'.
            mask_color_mode (str, optional): 'rgb' or 'grayscale'. Defaults to 'grayscale'.

        Raises:
            OSError: If image folder doesn't exist in root.
            OSError: If mask folder doesn't exist in root.
            ValueError: If subset is not either 'Train' or 'Test'
            ValueError: If image_color_mode and mask_color_mode are either 'rgb' or 'grayscale'
        """
        super().__init__(root, transforms)
        image_folder_path = Path(self.root) / image_folder
        mask_folder_path = Path(self.root) / mask_folder
        if not image_folder_path.exists():
            raise OSError(f"{image_folder_path} does not exist.")
        if not mask_folder_path.exists():
            raise OSError(f"{mask_folder_path} does not exist.")

        if image_color_mode not in ["rgb", "grayscale"]:
            raise ValueError(
                f"{image_color_mode} is an invalid choice. Please enter from rgb grayscale."
            )
        if mask_color_mode not in ["rgb", "grayscale"]:
            raise ValueError(
                f"{mask_color_mode} is an invalid choice. Please enter from rgb grayscale."
            )

        self.image_color_mode = image_color_mode
        self.mask_color_mode = mask_color_mode

        if not fraction:
            self.image_names = sorted(image_folder_path.glob("*"))
            self.mask_names = sorted(mask_folder_path.glob("*"))
        else:
            if subset not in ["Train", "Test"]:
                raise (ValueError(
                    f"{subset} is not a valid input. Acceptable values are Train and Test."
                ))
            self.fraction = fraction
            self.image_list = np.array(sorted(image_folder_path.glob("*")))
            self.mask_list = np.array(sorted(mask_folder_path.glob("*")))
            if seed:
                np.random.seed(seed)
                indices = np.arange(len(self.image_list))
                np.random.shuffle(indices)
                self.image_list = self.image_list[indices]
                self.mask_list = self.mask_list[indices]
            if subset == "Train":
                self.image_names = self.image_list[:int(
                    np.ceil(len(self.image_list) * (1 - self.fraction)))]
                self.mask_names = self.mask_list[:int(
                    np.ceil(len(self.mask_list) * (1 - self.fraction)))]
            else:
                self.image_names = self.image_list[
                    int(np.ceil(len(self.image_list) * (1 - self.fraction))):]
                self.mask_names = self.mask_list[
                    int(np.ceil(len(self.mask_list) * (1 - self.fraction))):]

    def __len__(self) -> int:
        return len(self.image_names)

    def __getitem__(self, index: int) -> Any:
        image_path = self.image_names[index]
        mask_path = self.mask_names[index]
        with open(image_path, "rb") as image_file, open(mask_path,
                                                        "rb") as mask_file:
            image = Image.open(image_file)
            if self.image_color_mode == "rgb":
                image = image.convert("RGB")
            elif self.image_color_mode == "grayscale":
                image = image.convert("L")
            mask = Image.open(mask_file)
            if self.mask_color_mode == "rgb":
                mask = mask.convert("RGB")
            elif self.mask_color_mode == "grayscale":
                mask = mask.convert("L")
            sample = {"image": image, "mask": mask}
            if self.transforms:
                sample["image"] = self.transforms(sample["image"])
                sample["mask"] = self.transforms(sample["mask"])
            return sample

In [ ]:
# Helper function to create dataloaders.
from pathlib import Path

from torch.utils.data import DataLoader
from torchvision import transforms


def get_dataloader_single_folder(data_dir: str,
                                 image_folder: str = 'Images',
                                 mask_folder: str = 'Masks',
                                 fraction: float = 0.2,
                                 batch_size: int = 4):
    """Create train and test dataloader from a single directory containing
    the image and mask folders.

    Args:
        data_dir (str): Data directory path or root
        image_folder (str, optional): Image folder name. Defaults to 'Images'.
        mask_folder (str, optional): Mask folder name. Defaults to 'Masks'.
        fraction (float, optional): Fraction of Test set. Defaults to 0.2.
        batch_size (int, optional): Dataloader batch size. Defaults to 4.

    Returns:
        dataloaders: Returns dataloaders dictionary containing the
        Train and Test dataloaders.
    """
    data_transforms = transforms.Compose([transforms.ToTensor()])

    image_datasets = {
        x: SegmentationDataset(data_dir,
                               image_folder=image_folder,
                               mask_folder=mask_folder,
                               seed=100,
                               fraction=fraction,
                               subset=x,
                               transforms=data_transforms)
        for x in ['Train', 'Test']
    }
    dataloaders = {
        x: DataLoader(image_datasets[x],
                      batch_size=batch_size,
                      shuffle=True,
                      num_workers=8)
        for x in ['Train', 'Test']
    }
    return dataloaders

In [ ]:
# Load pre-trained DeepLabV3 model with desired number of outputs.
def createDeepLabv3(outputchannels: bool = 1):
    """DeepLabv3 class with custom head
    Args:
        outputchannels (int, optional): The number of output channels
        in your dataset masks. Defaults to 1.
    Returns:
        model: Returns the DeepLabv3 model with the ResNet101 backbone.
    """
    model = models.segmentation.deeplabv3_resnet101(
        weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1,
        progress=False,
    )
    
    model.classifier = DeepLabHead(2048, outputchannels)
    # Set the model in training mode
    model.train()
    
    return model

In [ ]:
# Training function.
def train_model(model, criterion, dataloaders, optimizer, metrics, bpath,
                num_epochs):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    # Use gpu if available
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # Initialize the log file for training and testing loss and metrics
    fieldnames = ['epoch', 'Train_loss', 'Test_loss'] + \
        [f'Train_{m}' for m in metrics.keys()] + \
        [f'Test_{m}' for m in metrics.keys()]
    with open(os.path.join(bpath, 'log.csv'), 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

    for epoch in range(1, num_epochs + 1):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)
        # Each epoch has a training and validation phase
        # Initialize batch summary
        batchsummary = {a: [0] for a in fieldnames}

        for phase in ['Train', 'Test']:
            if phase == 'Train':
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            # Iterate over data.
            for sample in tqdm(iter(dataloaders[phase])):
                inputs = sample['image'].to(device)
                masks = sample['mask'].to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # track history if only in train
                with torch.set_grad_enabled(phase == 'Train'):
                    outputs = model(inputs)
                    loss = criterion(outputs['out'], masks)
                    y_pred = outputs['out'].data.cpu().numpy().ravel()
                    y_true = masks.data.cpu().numpy().ravel()
                    for name, metric in metrics.items():
                        if name == 'f1_score':
                            # Use a classification threshold of 0.1
                            batchsummary[f'{phase}_{name}'].append(
                                metric(y_true > 0, y_pred > 0.1))
                        else:
                            batchsummary[f'{phase}_{name}'].append(
                                metric(y_true.astype('uint8'), y_pred))

                    # backward + optimize only if in training phase
                    if phase == 'Train':
                        loss.backward()
                        optimizer.step()
            batchsummary['epoch'] = epoch
            epoch_loss = loss
            batchsummary[f'{phase}_loss'] = epoch_loss.item()
            print('{} Loss: {:.4f}'.format(phase, loss))
        for field in fieldnames[3:]:
            batchsummary[field] = np.mean(batchsummary[field])
        print(batchsummary)
        with open(os.path.join(bpath, 'log.csv'), 'a', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(batchsummary)
            # deep copy the model
            if phase == 'Test' and loss < best_loss:
                best_loss = loss
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Lowest Loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

## Create dataset to test example code.
There is not a lot of information of what the dataset should look like, but ideally it should be images and binary masks with positive class as either 1 or 255.

In [ ]:
# Create the directory of images and masks, put all on one.
data_dir = '/jcDataStore/Data/NeuroTK-Dash/ml-tissue-detection/DeepLabV3'

img_dir = join(data_dir, 'images')
mask_dir = join(data_dir, 'masks')

create_dirs([img_dir, mask_dir])

# Directory with original low res. WSIs and tissue masks.
# src_dir = '/jcDataStore/Data/nft-ai-project/wsi-inference/tissue-masks'
src_dir = '/jcDataStore/Data/NeuroTK-Dash/ml-tissue-detection/gray-matter-dataset'

for img_fp in tqdm(glob(join(src_dir, 'images', '*.png'))):
    # Skip if the image & mask have already been saved.
    fn = os.path.basename(img_fp)
    img_save_fp = join(img_dir, fn)
    mask_save_fp = join(mask_dir, fn)

    if  isfile(img_save_fp) and isfile(mask_save_fp):
        continue
        
    # Read the image and resize to standard size.
    img = imread(img_fp)
    img = cv.resize(img, (256, 256))

    # Repeat with the mask.
    mask = imread(join(src_dir, 'masks', fn))
    mask = mask[:, :, 0]  # Grab only one channel
    mask = cv.resize(mask, (256, 256))

    # Save in new locations.
    imwrite(img_save_fp, img)
    imwrite(mask_save_fp, mask)

## Test out Training

In [ ]:
# Get model.
model = createDeepLabv3()
_ = model.train()

In [ ]:
# Create location to save training results.
exp_dir = Path(
    '/jcDataStore/Data/NeuroTK-Dash/ml-tissue-detection/DeepLabV3/exps'
)
exp_dir.mkdir(exist_ok=True)

In [ ]:
# Specify the loss function
criterion = torch.nn.MSELoss(reduction='mean')

# Specify the optimizer with a lower learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Specify the evaluation metrics
metrics = {'f1_score': f1_score, 'auroc': roc_auc_score}

In [ ]:
# Get dataloaders.
data_directory = Path('/jcDataStore/Data/NeuroTK-Dash/ml-tissue-detection/gray-matter-dataset')

dataloaders = get_dataloader_single_folder(
    data_directory, batch_size=8, image_folder = 'images', 
    mask_folder='masks'
)

_ = train_model(
    model,
    criterion,
    dataloaders,
    optimizer,
    bpath=exp_dir,
    metrics=metrics,
    num_epochs=50
)

In [ ]:
# Read a sample image.
model.eval()

img = Image.open('/jcDataStore/Data/NeuroTK-Dash/ml-tissue-detection/gray-matter-dataset/images/A21-17_1_BIELS.png')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model.to('cpu')

for batch in dataloaders['Test']:
    img = batch['image'].to('cpu')
    mask = batch['mask'].to('cpu')

    # predict
    preds = model(img)['out']

    # Plot each
    for i in range(8):
        image = img[i, :, :, :].detach().numpy()
        image = np.moveaxis(image, 0, 2)
        m = mask[i, 0, :, :].detach().numpy()
        pred = preds[i, 0, :, :].detach().numpy() > 0.5

        fig = plt.figure(figsize=(7, 4))
        fig.add_subplot(1, 3, 1)
        plt.imshow(image)
        plt.axis('off')
        fig.add_subplot(1, 3, 2)
        plt.imshow(m)
        plt.title('Ground Truth Mask', fontsize=12)
        plt.axis('off')
        fig.add_subplot(1, 3, 3)
        plt.imshow(pred)
        plt.axis('off')
        plt.title('Prediction', fontsize=12)
        plt.show()
    break